# Lilsim Python SDK Testing

In [1]:
# Setup and imports
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Add the SDK to path
sys.path.insert(0, '.')

from lilsim import LilsimClient, ControlRequest, FrameId, ModelMetadata 
from lilsim.utils import state_to_dict

client = LilsimClient(host="localhost")
client.connect()

/home/will/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
INFO:lilsim.client:Connecting to lilsim at localhost...
INFO:lilsim.client:Connected to state stream (port 5556)
INFO:lilsim.client:Connected to admin command endpoint (port 5558)
INFO:lilsim.client:Connected to async control stream (port 5559)
INFO:lilsim.client:Connected to marker stream (port 5560)


In [2]:
metadata = client.refresh_metadata()
print(f"Loaded metadata for {metadata.model_name} (schema v{metadata.schema_version})")
print("Parameters:", [p.name for p in metadata.params])
print("Inputs:", [i.name for i in metadata.inputs])


INFO:lilsim.client:Admin command succeeded: Metadata attached


Loaded metadata for Kinematic Single Track (schema v5)
Parameters: ['wheelbase', 'track_width', 'v_max', 'steering_delay', 'drivetrain_delay', 'steering_rack_ratio']
Inputs: ['steering_wheel_angle_input', 'steering_wheel_rate_input', 'ax']


In [3]:
import numpy as np

current_state = None

def state_callback(state):
    global current_state
    current_state = client.decode_state_update(state)


def pursuit_point(s: float, sample_length: float, path_points: np.ndarray, 
    p_car: tuple[float, float], lookahead: float) -> tuple[float, float]:
    """ Return pure-pursuit given the position of the car.
    
        Input:
        s - current position on spline
        sample_length - length of spline sample
        path_points - points sampled from spline
        p_car - Car position in global coordinates
        
        Return:
        pursuit point in global coordinates
    """

    starting_index = int(s / sample_length)
    for i in range(starting_index, len(path_points)):
        d2 = np.linalg.norm(p_car - path_points[i])
        if d2 > lookahead:
            return path_points[i]
    
    return path_points[-1]

def pure_pursuit_control(dp, theta, wheelbase):
    """ Compute pure-pursuit steer angle.
    
        Input:
        dp - Vector from position of car to pursuit point
        theta - heading of vehicle
        
        Output:
        return steer angle
    """

    l = np.linalg.norm(dp)
    rot_mat = np.array([[np.cos(theta - np.pi/2), -np.sin(theta - np.pi/2)],
                        [np.sin(theta - np.pi/2),  np.cos(theta - np.pi/2)]])
    h_x = np.matmul(rot_mat, (np.array([[1, 0]]).reshape(-1, 1)))
    x = np.dot(dp, h_x)
    delta = np.arctan2(-wheelbase*2*x/(l**2), 1)

    return delta[0]
    
def pp_steer_angle(w: np.ndarray, pursuit_point: tuple[float, float], wheelbase: float):
    """ Compute control action
    
        Input:
        w - current state w = (x, y, theta, v)
        pursuit_point - pursuit point in global coordinates
        wheelbase - wheelbase of the car
        
        Output:
        return delta, the steer angle
    """
    x, y, theta, _ = w
    p_car = np.array([x, y])

    dp = pursuit_point - p_car
    delta = pure_pursuit_control(dp, theta, wheelbase)
    
    return delta

def acceleration_proportional(current_v: float, v_setpoint: float, Kp: float) -> float:
    ax = (v_setpoint - current_v) * Kp
    return ax

def visualize_projection(car_pos: tuple[float, float], spline_pos: tuple[float, float]):
    client.publish_line_strip(
        ns="projection", id=0, points=np.array([car_pos, spline_pos]), color=(255, 0, 0, 200), line_width=0.1
    )
    client.publish_circle(
        ns="projection", id=1, pos=car_pos, radius=0.2, color=(255, 0, 0, 200)
    )
    client.publish_circle(
        ns="projection", id=2, pos=spline_pos, radius=0.2, color=(255, 0, 0, 200)
    )

from splinepath import SplinePath
class PurePursuitController:
    def __init__(self, path, sample_length, wheelbase, lookahead, stopping_dist, v_max, Kp, steering_rack_ratio):
        self.spline_path = SplinePath(path)
        self.sample_length = sample_length
        self.spline_points = self.spline_path.sample(sample_length=sample_length) # resampling
        self.wheelbase = wheelbase
        self.lookahead = lookahead
        self.stopping_dist = stopping_dist
        self.v_max = v_max
        self.Kp = Kp
        self.steering_rack_ratio = steering_rack_ratio
        self.prev_s = 0.0

    def __name__(self):
        return "PurePursuitController"
    
    def __call__(self, request: ControlRequest, decoded_state: dict | None = None) -> tuple[float, float, float]:
        decoded = decoded_state or client.decode_scene_state(request.scene)
        states = decoded.get("states", {})
        x = states.get("x", 0.0)
        y = states.get("y", 0.0)
        theta = states.get("yaw", 0.0)
        v = states.get("v", 0.0)
        w = np.array([x, y, theta, v])

        s, _ = self.spline_path.project((x,y), self.prev_s)
        self.prev_s = s

        car_pos = (x, y)
        spline_projection_point = (self.spline_path.x(s), self.spline_path.y(s))
        visualize_projection(car_pos, spline_projection_point)

        pp_point = pursuit_point(s, self.sample_length, self.spline_points, car_pos, self.lookahead)
        client.publish_circle(
            ns="pure_pursuit", id=1, pos=pp_point, radius=0.2, color=(0, 255, 0, 200)
        )
        steer_angle = pp_steer_angle((x,y,theta,v), pp_point, self.wheelbase) * self.steering_rack_ratio
        
        v_setpoint = self.v_max if s < self.spline_path.length - self.stopping_dist else 0.0
        ax = acceleration_proportional(v, v_setpoint, self.Kp)
        # print(f"ax_prop(v={v}, v_setpoint={v_setpoint}, Kp={self.Kp}) = {ax}, at s = {s}")
        # print(f"steer_angle: {steer_angle}, ax: {ax}")

        return (steer_angle, 0.0, ax) # (steer_angle, steer_rate, ax)

    def visualize_spline(self, stopping_dist=5.0):
        # publish points sampled from spline path as a line strip
        n_points = self.spline_points.shape[0]
        colors = np.zeros((n_points, 4), dtype=np.uint8)
        colors[:, 0] = np.linspace(0, 255, n_points).astype(np.uint8)  # Red: 0 -> 255
        colors[:, 1] = np.linspace(0, 50, n_points).astype(np.uint8)  # Green: 0 -> 255
        colors[:, 2] = np.linspace(255, 120, n_points).astype(np.uint8)  # Blue: 255 -> 0
        colors[:, 3] = 255

        # color the last stopping_dist meters yellow
        stopping_dist_samples = int(stopping_dist / self.sample_length)
        colors[-stopping_dist_samples:, 0] = 255
        colors[-stopping_dist_samples:, 1] = 255
        colors[-stopping_dist_samples:, 2] = 0
        
        client.publish_line_strip(
            ns="spline", id=0, frame_id=FrameId.WORLD, points=self.spline_points, colors=colors, line_width=0.1
        )

    def reset(self):
        self.prev_s = 0.0


In [4]:
# load midpoints
track = '/home/will/code/lilsim/tracks/skidpad.csv'
data = np.genfromtxt(track, delimiter=',', skip_header=1, dtype=str)
midpoint_rows = data[data[:,0] == 'midpoint']
midpoint_path = midpoint_rows[:,1:3].astype(float)

# fetch steering_rack_ratio
steering_rack_ratio = client.get_car_parameter("steering_rack_ratio")
print(f"steering_rack_ratio: {steering_rack_ratio}")

# controller params
L = 1.9
spline_sample_length = 0.5 # for spline resampling
lookahead = 4.0
stopping_dist = 10.0
v_max = 15.0
prev_s = 0.0
Kp = 5.0
controller = PurePursuitController(midpoint_path, spline_sample_length, L, lookahead, stopping_dist, v_max, Kp, steering_rack_ratio)

client.clear_markers()
controller.visualize_spline(stopping_dist)

# visualize lookahead ring
client.publish_ring(
    ns="pure_pursuit", id=0, frame_id=FrameId.CAR, pos=(0, 0), radius=lookahead, color=(255, 100, 255, 200)
)

client.set_parameters({
    "wheelbase": L,
    "v_max": v_max,
})
client.set_settings({"steering_input_mode": 0})
client.set_track(track)
client.set_simulation_config(timestep_ms=1.0, control_period_ms=10.0, control_delay_ms=2.0)
client.set_control_mode(sync=True, external_control=True)
client.register_sync_controller(controller)
client.reset()  # reset simulator (paused at start)
controller.reset()  # reset controller state
client.start()  # start communication threads


INFO:lilsim.client:Admin command succeeded: Parameter overrides staged
INFO:lilsim.client:Admin command succeeded: Setting overrides staged
INFO:lilsim.client:Admin command succeeded: Track loaded
INFO:lilsim.client:Admin command succeeded: Simulation config updated


steering_rack_ratio: 4.6


INFO:lilsim.client:Admin command succeeded: Sync mode, source=ZeroMQ client
INFO:lilsim.client:Registered sync controller: <bound method PurePursuitController.__name__ of <__main__.PurePursuitController object at 0x77e1dbd1a440>>
INFO:lilsim.client:Connected to sync control endpoint (port 5557)
INFO:lilsim.client:Admin command succeeded: Reset requested
INFO:lilsim.client:State listener thread started
INFO:lilsim.client:Control responder thread started
INFO:lilsim.client:Client started


INFO:lilsim.client:Admin command succeeded: Metadata attached


In [5]:
client.reset() # reset simulator (p used at start)
controller.reset() # reset controller state
client.run()   # resume simulation loop

INFO:lilsim.client:Admin command succeeded: Reset requested
INFO:lilsim.client:Admin command succeeded: Running


True

In [ ]:
client.stop()  # stop communication threads

In [ ]:
# car marker example
wheelbase = client.get_car_parameter("wheelbase")
track_width = client.get_car_parameter("track_width")
client.publish_car_marker(
    ns="demo",
    id=0,
    pose=(1.0, 2.0, 0.4),
    wheelbase=wheelbase,
    track_width=track_width,
    wheel_fl_angle=0.15,
    wheel_fr_angle=0.15,
    opacity=0.6,
    tint_color=(255, 50, 0, 255),
    tint_opacity=0.5,
)


In [ ]:
client.clear_markers()

In [ ]:
client.pause()
client.stop()